# Loading in the Data & Basic EDA

### This file is to do EDA and basic cleaning/transformation

In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline

#modeling
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
#from sklearn import metrics

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score
from sklearn.model_selection import learning_curve

#Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB

#SVM (Support Vector Machine) Classifier
from sklearn.svm import SVC

#Decision Tree
from sklearn.tree import DecisionTreeClassifier

#Random Forest
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score, roc_curve


In [2]:
weather_list = [
"ALBANY",
"BROOME",
"CATTARAUGUS",
"CLINTON",
"DUTCHESS",
"ERIE",
"ESSEX",
"FULTON",
"GREENE",
"JEFFERSON",
"MONROE",
"NASSAU",
"ONEIDA",
"ONONDAGA",
"ONTARIO",
"ORANGE",
"OTSEGO",
"RENSSELAER",
"ROCKLAND",
"SARATOGA",
"SCHENECTADY",
"SUFFOLK",
"ULSTER",
"WESTCHESTER"]

In [3]:
df = pd.read_csv('../MV_crashes.csv')

In [4]:
df.Year.value_counts()

2016    301908
2014    299452
2015    294556
Name: Year, dtype: int64

In [5]:
#Remove NYC counties
#NYC = ["QUEENS","KINGS", "NEW YORK", "BRONX", "RICHMOND"]
#df = df[~df["County Name"].isin(NYC)].reset_index()

#Select countys with weather station data
df = df[df["County Name"].isin(weather_list)].reset_index()
print("Shape of data set: ", df.shape)

Shape of data set:  (559561, 19)


In [6]:
#drop cols dont need
df.drop(['Year', 'Police Report', 'Municipality', 'DOT Reference Marker Location', 
         'Pedestrian Bicyclist Action', 'Pedestrian Bicyclist Action', 'Collision Type Descriptor'] 
         ,axis=1, inplace=True)

In [7]:
print("Counties being used")
print(df['County Name'].unique())
print("\n")
print(df['County Name'].value_counts())

Counties being used
['ERIE' 'SUFFOLK' 'GREENE' 'JEFFERSON' 'ULSTER' 'NASSAU' 'ORANGE'
 'CATTARAUGUS' 'DUTCHESS' 'RENSSELAER' 'CLINTON' 'WESTCHESTER'
 'SCHENECTADY' 'ROCKLAND' 'ESSEX' 'ALBANY' 'MONROE' 'ONTARIO' 'ONONDAGA'
 'BROOME' 'SARATOGA' 'ONEIDA' 'OTSEGO' 'FULTON']


NASSAU         98983
SUFFOLK        96013
ERIE           51699
WESTCHESTER    44364
MONROE         42096
ONONDAGA       29168
ORANGE         28856
ALBANY         24541
ROCKLAND       19745
DUTCHESS       17195
SARATOGA       14156
ONEIDA         13209
ULSTER         12899
BROOME         11791
SCHENECTADY     9857
RENSSELAER      8977
ONTARIO         7336
JEFFERSON       6228
CLINTON         5153
CATTARAUGUS     4490
OTSEGO          3693
GREENE          3535
ESSEX           2878
FULTON          2699
Name: County Name, dtype: int64


In [8]:
df['Day of Week'].value_counts()

Friday       93314
Thursday     85673
Wednesday    84382
Tuesday      83247
Monday       80662
Saturday     72475
Sunday       59808
Name: Day of Week, dtype: int64

# Transforming & Mapping Data

### Changing Crash Descriptor ie Y

In [9]:
df['Crash Descriptor'].value_counts(normalize=True)

Property Damage Accident             0.663631
Property Damage & Injury Accident    0.220950
Injury Accident                      0.112417
Fatal Accident                       0.003002
Name: Crash Descriptor, dtype: float64

Lets map everything involving any sort of injury as the injury class

In [10]:
#property damage = 0
#injury = 1 
crash_map = {"Property Damage Accident": 0,
             "Property Damage & Injury Accident": 1,
             "Injury Accident": 1,
            "Fatal Accident" : 1}
df['Crash Descriptor'].replace(crash_map, inplace=True)

In [11]:
#end up with 66/33 split
##Non-Injury is class 0 / Injury is class 1
df['Crash Descriptor'].value_counts(normalize=True)

0    0.663631
1    0.336369
Name: Crash Descriptor, dtype: float64

### Transform Time varaible to extract Hour / Transform Date to extract month

In [12]:
#get hour of the day 
df["Hour"] = df["Time"].str.split(':').str[0].astype(int)

#get month from date
df["Month"] = df["Date"].str.split('-').str[1].astype(int)
df['Date'] = df["Date"].str.split('T').str[0] #split seconds out of Date field to get just the date

### Road Descriptor transform

In [13]:
df["Road Descriptor"].value_counts()

Straight and Level        397312
Straight and Grade         61210
Curve and Level            35643
Unknown                    27392
Curve and Grade            26974
Straight at Hill Crest      8185
Curve at Hill Crest         2845
Name: Road Descriptor, dtype: int64

In [14]:
#Neither represents unknown cases. Ie drop_first
#create 2 columns for straight or curve
df['Straight_Road'] = df["Road Descriptor"].str.contains("Straight").astype(int)
df['Curve_Road'] = df["Road Descriptor"].str.contains("Curve").astype(int)

In [15]:
#create 3 columns from street slope
df['Level_Road'] = df["Road Descriptor"].str.contains("Level").astype(int)
df['Grade_Road'] = df["Road Descriptor"].str.contains("Grade").astype(int)
df['Hill_Road'] = df["Road Descriptor"].str.contains("Hill Crest").astype(int)
df.drop(["Road Descriptor"], axis=1, inplace=True) #drop original column when done with it

In [16]:
#print out df to see changes made
df.head(3)

,index,Crash Descriptor,Time,Date,Day of Week,Lighting Conditions,County Name,Weather Conditions,Traffic Control Device,Road Surface Conditions,Event Descriptor,Number of Vehicles Involved,Hour,Month,Straight_Road,Curve_Road,Level_Road,Grade_Road,Hill_Road
0,3,0,12:32,2016-12-31,Saturday,Daylight,ERIE,Snow,None,Snow/Ice,"Snow Embankment, Collision With Fixed Object",1,12,12,1,0,1,0,0
1,5,1,15:40,2016-12-31,Saturday,Daylight,SUFFOLK,Clear,Traffic Signal,Dry,"Other Motor Vehicle, Collision With",2,15,12,1,0,1,0,0
2,6,0,20:15,2016-12-31,Saturday,Dark-Road Unlighted,GREENE,Cloudy,None,Wet,Deer,1,20,12,1,0,1,0,0


### Traffic Signal

In [17]:
df['Traffic Control Device'].value_counts()

None                                      317953
Traffic Signal                            111655
Stop Sign                                  48173
Unknown                                    33821
No Passing Zone                            31635
Yield Sign                                  5930
Other                                       4014
Construction Work Area                      1976
Flashing Light                              1409
RR Crossing Gates                            680
Not Applicable                               507
Police/Fire Emergency                        437
Stopped School Bus-Red Lights Flashing       402
Officer/Guard                                246
Maintenance Work Area                        244
School Zone                                  196
RR Crossing Sign                             130
Utility Work Area                             96
RR Crossing Flashing Light                    57
Name: Traffic Control Device, dtype: int64

In [18]:
#assumption: if not specified it did not occur at a traffice device/sign
traffic_dev_map = {
"None": "No Traffic Device",
"Traffic Signal": "Traffic Device",
"Stop Sign":      "Traffic Device",
"No Passing Zone": "No Traffic Device",
"Unknown": "Unknown",             #Assumptions made
"Yield Sign": "Traffic Device",
"Other": "No Traffic Device",               #Assumptions made
"Construction Work Area": "No Traffic Device",
"Flashing Light": "Traffic Device", 
"RR Crossing Gates": "Traffic Device",
"Not Applicable": "No Traffic Device",
"Police/Fire Emergency":"No Traffic Device",
"Stopped School Bus-Red Lights Flashing": "Traffic Device",
"Maintenance Work Area": "No Traffic Device",
"Officer/Guard": "No Traffic Device",
"School Zone": "No Traffic Device",
"RR Crossing Sign": "Traffic Device",
"Utility Work Area": "No Traffic Device",
"RR Crossing Flashing Light": "No Traffic Device"
}

In [19]:
#Apply dictionary map
df['Traffic Control Device'].replace(traffic_dev_map, inplace=True)

In [20]:
df.head(2)

,index,Crash Descriptor,Time,Date,Day of Week,Lighting Conditions,County Name,Weather Conditions,Traffic Control Device,Road Surface Conditions,Event Descriptor,Number of Vehicles Involved,Hour,Month,Straight_Road,Curve_Road,Level_Road,Grade_Road,Hill_Road
0,3,0,12:32,2016-12-31,Saturday,Daylight,ERIE,Snow,No Traffic Device,Snow/Ice,"Snow Embankment, Collision With Fixed Object",1,12,12,1,0,1,0,0
1,5,1,15:40,2016-12-31,Saturday,Daylight,SUFFOLK,Clear,Traffic Device,Dry,"Other Motor Vehicle, Collision With",2,15,12,1,0,1,0,0


### Event Descriptor

In [21]:
#Types of collisions in the data set
print(df['Event Descriptor'].nunique())
df['Event Descriptor'].value_counts()

33


Other Motor Vehicle, Collision With                             401119
Deer                                                             36112
Guide Rail - Not At End, Collision With Fixed Object             16736
Light Support/Utility Pole, Collision With Fixed Object          12966
Tree, Collision With Fixed Object                                11889
Pedestrian, Collision With                                       11785
Earth Embankment/Rock Cut/Ditch, Collision With Fixed Object     10778
Other Fixed Object*, Collision With Fixed Object                  5912
Bicyclist, Collision With                                         5665
Animal, Collision With                                            5533
Other Object (Not Fixed)*, Collision With                         5222
Sign Post, Collision With Fixed Object                            5207
Curbing, Collision With Fixed Object                              4194
Building/Wall, Collision With Fixed Object                        3799
Snow E

In [22]:
event_mapper = {
"Other Motor Vehicle, Collision With": "Vehicle Collision",
"Deer": "Animal Collision",
"Guide Rail - Not At End, Collision With Fixed Object": "Fixed Object Collision",
"Earth Embankment/Rock Cut/Ditch, Collision With Fixed Object": "Fixed Object Collision",
"Light Support/Utility Pole, Collision With Fixed Object": "Fixed Object Collision",
"Tree, Collision With Fixed Object": "Fixed Object Collision",
"Pedestrian, Collision With": "Pedestrian Collision",
"Animal, Collision With": "Animal Collision",
"Other Fixed Object*, Collision With Fixed Object": "Fixed Object Collision",
"Sign Post, Collision With Fixed Object": "Fixed Object Collision",
"Other Object (Not Fixed)*, Collision With": "Non-Fixed Object Collision", 
"Bicyclist, Collision With": "Bike Collision",
"Curbing, Collision With Fixed Object": "Fixed Object Collision",
"Building/Wall, Collision With Fixed Object": "Fixed Object Collision",
"Snow Embankment, Collision With Fixed Object": "Fixed Object Collision",
"Other*, Non-Collision": "Non-Collision",
"Overturned, Non-Collision": "Non-Collision",
"Barrier, Collision With Fixed Object": "Fixed Object Collision",
"Fence, Collision With Fixed Object": "Fixed Object Collision",
"Guide Rail - End, Collision With Fixed Object": "Fixed Object Collision",
"Ran Off Roadway Only, Non-Collision": "Non-Collision",
"Median - Not At End, Collision With Fixed Object": "Fixed Object Collision",
"Bridge Structure, Collision With Fixed Object": "Fixed Object Collision",
"Fire Hydrant, Collision With Fixed Object": "Fixed Object Collision",
"Culver/Head Wall, Collision With Fixed Object": "Fixed Object Collision",
"Fire/Explosion, Non-Collision": "Non-Collision",
"Unknown": "Unknown",
"Crash Cushion, Collision With Fixed Object": "Fixed Object Collision",
"Other Pedestrian": "Pedestrian Collision",
"Median - End, Collision With Fixed Object": "Fixed Object Collision",
"Submersion, Non-Collision": "Non-Collision",
"Railroad Train, Collision With": "Fixed Object Collision",
"In-Line Skater, Collision With": "Pedestrian Collision"
}

In [23]:
#Map collision type
df['Event Descriptor'].replace(event_mapper, inplace=True)

In [24]:
#show target level breakdown for each target
f = df.groupby(['Event Descriptor', 'Crash Descriptor'])['Crash Descriptor'].count()
print(f)

Event Descriptor            Crash Descriptor
Animal Collision            0                    39739
                            1                     1906
Bike Collision              0                      100
                            1                     5565
Fixed Object Collision      0                    58311
                            1                    27854
Non-Collision               0                     3831
                            1                     3414
Non-Fixed Object Collision  0                     4291
                            1                      931
Pedestrian Collision        0                       54
                            1                    11957
Unknown                     0                      363
                            1                      126
Vehicle Collision           0                   264653
                            1                   136466
Name: Crash Descriptor, dtype: int64


# Join weather data here before using doing weather varaibles

In [25]:
df.head(2)

,index,Crash Descriptor,Time,Date,Day of Week,Lighting Conditions,County Name,Weather Conditions,Traffic Control Device,Road Surface Conditions,Event Descriptor,Number of Vehicles Involved,Hour,Month,Straight_Road,Curve_Road,Level_Road,Grade_Road,Hill_Road
0,3,0,12:32,2016-12-31,Saturday,Daylight,ERIE,Snow,No Traffic Device,Snow/Ice,Fixed Object Collision,1,12,12,1,0,1,0,0
1,5,1,15:40,2016-12-31,Saturday,Daylight,SUFFOLK,Clear,Traffic Device,Dry,Vehicle Collision,2,15,12,1,0,1,0,0


In [26]:
#bring in weather data. This has been gotten from the 
df_weather = pd.read_csv('weather.csv', usecols=['County_join','DATE_TIME_PROCESSED',
                                                 'T_HIGH_F','WEATHER_DESC','WIND_MPH'])

In [27]:
df_weather['County_join'].value_counts()

ERIE         916
NASSAU       916
MONROE       915
ONONDAGA     915
SUFFOLK      912
DUTCHESS     905
ALBANY       904
BROOME       904
ORANGE       902
FULTON       902
ESSEX        893
CLINTON      890
JEFFERSON    881
Name: County_join, dtype: int64

In [28]:
#find position to add this in and a place to join this on date
county_weather_map = { 
"ESSEX" : "ESSEX",
"FULTON" : "FULTON",
"RENSSELAER" : "FULTON",
"BROOME" : "BROOME",
"ALBANY" : "ALBANY",
"DUTCHESS" : "DUTCHESS",
"GREENE" : "DUTCHESS",
"ULSTER" : "DUTCHESS",
"JEFFERSON" : "JEFFERSON",
"ERIE" : "ERIE",
"MONROE" : "MONROE",
"ONTARIO" : "MONROE",
"NASSAU" : "NASSAU",
"WESTCHESTER" : "NASSAU",
"ORANGE" : "ORANGE",
"CLINTON" : "CLINTON",
"ONONDAGA" : "ONONDAGA",
"SUFFOLK" : "SUFFOLK",
#below are mapped to nearest station
"ONEIDA" : "ONONDAGA",
"OTSEGO" : "BROOME",
"ROCKLAND" : "NASSAU",
"CATTARAUGUS" : "ERIE",
"SARATOGA" : "ALBANY",
"SCHENECTADY" : "ALBANY"
}

In [31]:
#add a column to use as joining since there are multiple Counties w/ same weather station
df["County_join"] = df['County Name'].replace(county_weather_map) 
df_orig = df.copy() #copy of original to save incase of errors
df_comb = pd.merge(df, df_weather, how='left', left_on=['County_join', 'Date'], 
               right_on=['County_join', 'DATE_TIME_PROCESSED'])

In [32]:
#Using the mean and mode by county by month to fill in NaNs

print(df_comb.isna().sum().tail(4))

#Sorting just to make sure order is fine
df_comb.sort_values(by=["Date", "County Name"], inplace=True)

#fill with county level monthly average (best could do)
df_comb['WIND_MPH'] = df_comb['WIND_MPH'].fillna(df_comb.groupby(['County Name', 'Month'])['WIND_MPH'].transform('mean'))
df_comb['T_HIGH_F'] = df_comb['T_HIGH_F'].fillna(df_comb.groupby(['County Name', 'Month'])['T_HIGH_F'].transform('mean'))
df_comb['WEATHER_DESC'] = df_comb['WEATHER_DESC'].fillna(df_comb.groupby(['County Name', 'Month'])['WEATHER_DESC'].apply(lambda x: x.fillna(x.mode()[0])))                                                        
print(df_comb.isna().sum().tail(4))


WEATHER_DESC           91711
DATE_TIME_PROCESSED    91708
T_HIGH_F               91708
WIND_MPH               91708
dtype: int64
WEATHER_DESC               0
DATE_TIME_PROCESSED    91708
T_HIGH_F                   0
WIND_MPH                   0
dtype: int64


'\n'

In [34]:
#extracting weather conditions from weather description
df_comb['Rainy Road?'] = df_comb["WEATHER_DESC"].str.lower().str.contains("rain|drizzle|shower|storm").astype(int)
df_comb['Foggy Road?'] = df_comb["WEATHER_DESC"].str.lower().str.contains("fog").astype(int)
df_comb['Icy Road?'] = df_comb["WEATHER_DESC"].str.lower().str.contains("ice|snow|freeze").astype(int)
df_comb['Clear Conditions?'] = df_comb["WEATHER_DESC"].str.lower().str.contains("clear").astype(int)
df_comb['Sunny Conditions?'] = df_comb["WEATHER_DESC"].str.lower().str.contains("sunny").astype(int)
df_comb['Cloudy Conditions?'] = df_comb["WEATHER_DESC"].str.lower().str.contains("cloud|overcast").astype(int)

In [35]:
#Current number of columns in the DF
df_comb.columns

Index(['index', 'Crash Descriptor', 'Time', 'Date', 'Day of Week',
       'Lighting Conditions', 'County Name', 'Weather Conditions',
       'Traffic Control Device', 'Road Surface Conditions', 'Event Descriptor',
       'Number of Vehicles Involved', 'Hour', 'Month', 'Straight_Road',
       'Curve_Road', 'Level_Road', 'Grade_Road', 'Hill_Road', 'County_join',
       'WEATHER_DESC', 'DATE_TIME_PROCESSED', 'T_HIGH_F', 'WIND_MPH',
       'Rainy Road?', 'Foggy Road?', 'Icy Road?', 'Clear Conditions?',
       'Sunny Conditions?', 'Cloudy Conditions?'],
      dtype='object')

In [36]:
df_comb.head(2)

,index,Crash Descriptor,Time,Date,Day of Week,Lighting Conditions,County Name,Weather Conditions,Traffic Control Device,Road Surface Conditions,...,WEATHER_DESC,DATE_TIME_PROCESSED,T_HIGH_F,WIND_MPH,Rainy Road?,Foggy Road?,Icy Road?,Clear Conditions?,Sunny Conditions?,Cloudy Conditions?
559266,895339,0,18:53,2014-01-01,Wednesday,Dark-Road Unlighted,ALBANY,Cloudy,No Traffic Device,Dry,...,Passing clouds.,2014-01-01,18.0,2.486,0,0,0,0,0,1
559273,895352,0,2:36,2014-01-01,Wednesday,Dark-Road Lighted,ALBANY,Cloudy,No Traffic Device,Wet,...,Passing clouds.,2014-01-01,18.0,2.486,0,0,0,0,0,1


### Final Prep

In [37]:
df_use = df_comb.copy()
df_use.reset_index(inplace=True)
#dopping all of the columns used for feature engineering and joining
df_use.drop(['Weather Conditions', 'Lighting Conditions', 
             'Road Surface Conditions', 'Date','DATE_TIME_PROCESSED','WEATHER_DESC', 'Time','County_join',
             'level_0', 'index'] ,axis=1, inplace=True)

In [38]:
df_use.head(3)

,Crash Descriptor,Day of Week,County Name,Traffic Control Device,Event Descriptor,Number of Vehicles Involved,Hour,Month,Straight_Road,Curve_Road,...,Grade_Road,Hill_Road,T_HIGH_F,WIND_MPH,Rainy Road?,Foggy Road?,Icy Road?,Clear Conditions?,Sunny Conditions?,Cloudy Conditions?
0,0,Wednesday,ALBANY,No Traffic Device,Non-Fixed Object Collision,1,18,1,0,1,...,0,0,18.0,2.486,0,0,0,0,0,1
1,0,Wednesday,ALBANY,No Traffic Device,Vehicle Collision,2,2,1,1,0,...,0,0,18.0,2.486,0,0,0,0,0,1
2,0,Wednesday,ALBANY,No Traffic Device,Vehicle Collision,2,2,1,1,0,...,0,0,18.0,2.486,0,0,0,0,0,1


In [39]:
df_dummied = pd.get_dummies(df_use)

print("The number of columns now is: ", df_dummied.shape[1])
print(df_dummied.columns)

The number of columns now is:  59
Index(['Crash Descriptor', 'Number of Vehicles Involved', 'Hour', 'Month',
       'Straight_Road', 'Curve_Road', 'Level_Road', 'Grade_Road', 'Hill_Road',
       'T_HIGH_F', 'WIND_MPH', 'Rainy Road?', 'Foggy Road?', 'Icy Road?',
       'Clear Conditions?', 'Sunny Conditions?', 'Cloudy Conditions?',
       'Day of Week_Friday', 'Day of Week_Monday', 'Day of Week_Saturday',
       'Day of Week_Sunday', 'Day of Week_Thursday', 'Day of Week_Tuesday',
       'Day of Week_Wednesday', 'County Name_ALBANY', 'County Name_BROOME',
       'County Name_CATTARAUGUS', 'County Name_CLINTON',
       'County Name_DUTCHESS', 'County Name_ERIE', 'County Name_ESSEX',
       'County Name_FULTON', 'County Name_GREENE', 'County Name_JEFFERSON',
       'County Name_MONROE', 'County Name_NASSAU', 'County Name_ONEIDA',
       'County Name_ONONDAGA', 'County Name_ONTARIO', 'County Name_ORANGE',
       'County Name_OTSEGO', 'County Name_RENSSELAER', 'County Name_ROCKLAND',
       

In [40]:
df_dummied.head(2)

,Crash Descriptor,Number of Vehicles Involved,Hour,Month,Straight_Road,Curve_Road,Level_Road,Grade_Road,Hill_Road,T_HIGH_F,...,Traffic Control Device_Traffic Device,Traffic Control Device_Unknown,Event Descriptor_Animal Collision,Event Descriptor_Bike Collision,Event Descriptor_Fixed Object Collision,Event Descriptor_Non-Collision,Event Descriptor_Non-Fixed Object Collision,Event Descriptor_Pedestrian Collision,Event Descriptor_Unknown,Event Descriptor_Vehicle Collision
0,0,1,18,1,0,1,1,0,0,18.0,...,0,0,0,0,0,0,1,0,0,0
1,0,2,2,1,1,0,1,0,0,18.0,...,0,0,0,0,0,0,0,0,0,1


In [41]:
#final step saving CSV for other notebook
df_dummied.to_csv('final.csv', header= True, index=False)